# Middleware
Middleware 是插入 Agent 执行流程的 可插拔组件 ，用于在关键节点拦截、修改或增强数据流，实现 动态提示 、 状态管理 、 错误处理 、 工具控制 等功能。

LangChain 中间件的设计就是 “即插即用” ，开发者在使用中间件的过程中，只需要关注中间件的功能与需求是否匹配，不需要关心中间件调用的位置。

`middleware` 参数是专为 `langchain.agents.create_agent()` 设计的，LangGraph 用节点 + 边替代了中间件，提供更细粒度控制和更好的性能

<img src="./assets/middleware.png" width="800">

LangChain 中间件有 6 种类型钩子 ，分为 节点式（Node-style） 和 包装式（Wrap-style） 两大类。

1. 节点式钩子（Node-style Hooks）

顺序执行，用于日志、验证、状态更新。

|钩子名	|执行时机	|用途|
|----|----|----|
|beforeAgent	|Agent 开始前（一次）	|加载状态、验证输入|
|beforeModel	|模型调用前（每次）	|修改提示、裁剪消息|
|afterModel	|模型调用后（每次）	|验证输出、防护栏|
|afterAgent	|Agent 结束后（一次）	|保存结果、清理|

2. 包装式钩子（Wrap-style Hooks）

嵌套执行，控制 handler 调用次数（0/1 / 多次）。
|钩子名	|执行时机	|用途|
|----|----|----|
|wrapModelCall|	包装模型调用|	重试、缓存、转换|
|wrapToolCall	|包装工具调用|	错误处理、权限|

# Dynamic Prompt

静态提示词无法适应不同的用户需求和运行时上下文，而动态提示词让你根据实际情况实时调整 LLM 的行为。

想象你有一个客服助手。如果你用静态提示词，每个用户都会得到完全相同的回复风格。但实际上：

- 专家用户想要深入的技术细节
- 新手用户需要简化的解释
- VIP 用户可能需要更高的优先级处理

动态提示词就是在这一刻决定如何指导 LLM。
<img src="./assets/DynamicPrompts.png" width="800">

## 初始化
加载并检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


In [19]:
#导入实例数据库
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"
    

SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""


## 构建动态提示词
利用运行时上下文和中间件来生成动态提示词。

In [20]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

在 `create_agent` 中加入中间件。

In [21]:

from langchain_qwq import ChatQwen
import os
model=ChatQwen(
    model="qwen3-max", 
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    )

from langchain.agents import create_agent

agent=create_agent(
    model=model,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [22]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

I need to find the most costly purchase by Frank Harris. However, looking at the available tables (Album, Artist, Genre, Playlist, PlaylistTrack, Track), there is no table that contains customer information or purchase data. The database appears to be a music library database without sales or customer tables.

Therefore, I cannot determine the most costly purchase by Frank Harris with the given tables.


In [23]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

To find the most costly purchase by Frank Harris, I will:

1. Identify tables that may contain customer and purchase information.
2. Locate purchases associated with "Frank Harris."
3. Determine the most costly among them.

First, I'll inspect the database schema to understand the structure.
Tool Calls:
  execute_sql (call_e364a860a6ef4b84a4fe2b19)
 Call ID: call_e364a860a6ef4b84a4fe2b19
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ================